# Импорт библиотек

In [ ]:
import os  
import pandas as pd  
import spacy  # для токенизатора
import torch
from torch.nn.utils.rnn import pad_sequence  
from torch.utils.data import DataLoader, Dataset
from PIL import Image  
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.optim as optim
import statistics
import torchvision.models as models

from tqdm import tqdm
import torchvision.transforms as transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Эмбэддинг слов

*Мы хотим преобразовать текст в числовые значения*
1. Нам нужен словарь, сопоставляющий каждое слово индексу
2. Нам нужно настроить набор данных Pytorch для загрузки данных
3. Настройка заполнения каждого батча (все предложения должны быть одной длины)

In [ ]:
spacy_eng = spacy.load("en")

***Токенезация предложений*** 

Токенезация = векторизация

Помимо эмбеддинга слов в вектора, используют специальные токены:

* `PAD` - используется для выравни вания предложений, если предложение больше установленной длины, слова обрезаются, если меньше - в предложение добавляется pad стольео раз, пока длина не достигнет необходимого значения

* `SOS` - "start of sentence" то же, что и <start> на картинке ниже - первый токен, который подается в декодер вместе с вектором, чтобы начать генерировать токены ответа
* `EOS` - "end of sentence" - то же, что и <end> на картинке ниже - как только декодер сгенерирует этот токен, мы считаем ответ полным (обычные знаки препинания для этой цели использовать нельзя, так как их значениея могут быть разные)

* `UNK` - "unknown token" - используется для замены редких слов, которые не вошли к ваш словарь. Например, предложение «Меня зовут guotong1999» будет переведено на «Меня зовут _unk_»

<img src='https://cloud.githubusercontent.com/assets/2272790/18410099/1d0a1c1a-7761-11e6-9fe1-bd2e5622b90a.png'>

In [ ]:
# Создание словаря
class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

#  Загрузка Flickr8k датасета

In [ ]:
!unzip drive/MyDrive/flickr8k.zip

Streaming output truncated to the last 5000 lines.
  inflating: flickr8k/images/2844846111_8c1cbfc75d.jpg  
  inflating: __MACOSX/flickr8k/images/._2844846111_8c1cbfc75d.jpg  
  inflating: flickr8k/images/1082252566_8c79beef93.jpg  
  inflating: __MACOSX/flickr8k/images/._1082252566_8c79beef93.jpg  
  inflating: flickr8k/images/1386251841_5f384a0fea.jpg  
  inflating: __MACOSX/flickr8k/images/._1386251841_5f384a0fea.jpg  
  inflating: flickr8k/images/2196316998_3b2d63f01f.jpg  
  inflating: __MACOSX/flickr8k/images/._2196316998_3b2d63f01f.jpg  
  inflating: flickr8k/images/3106791484_13e18c33d8.jpg  
  inflating: __MACOSX/flickr8k/images/._3106791484_13e18c33d8.jpg  
  inflating: flickr8k/images/3603064161_a8f3b6455d.jpg  
  inflating: __MACOSX/flickr8k/images/._3603064161_a8f3b6455d.jpg  
  inflating: flickr8k/images/2293149170_38fb2257ea.jpg  
  inflating: __MACOSX/flickr8k/images/._2293149170_38fb2257ea.jpg  
  inflating: flickr8k/images/2451114871_8617ae2f16.jpg  
  inflating: __MA

In [ ]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform

        # Сохраняем картинку и описание
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]

        # Создаем словарь
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)



# Приводим описание картинок и их эмбеддинги к общему виду
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets


def get_loader(
    root_folder,
    annotation_file,
    transform,
    batch_size=32,
    num_workers=8,
    shuffle=True,
    pin_memory=True,
):
    dataset = FlickrDataset(root_folder, annotation_file, transform=transform)

    pad_idx = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_idx=pad_idx),
    )

    return loader, dataset



transform = transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor(),]
    )

loader, dataset = get_loader(
        "/content/flickr8k/images/", "/content/flickr8k/captions.txt", transform=transform
    )

for idx, (imgs, captions) in enumerate(loader):
        print(imgs.shape)
        print(captions.shape)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([32, 3, 224, 224])
torch.Size([21, 32])
torch.Size([32, 3, 224, 224])
torch.Size([22, 32])
torch.Size([32, 3, 224, 224])
torch.Size([21, 32])
torch.Size([32, 3, 224, 224])
torch.Size([22, 32])
torch.Size([32, 3, 224, 224])
torch.Size([28, 32])
torch.Size([32, 3, 224, 224])
torch.Size([22, 32])
torch.Size([32, 3, 224, 224])
torch.Size([20, 32])
torch.Size([32, 3, 224, 224])
torch.Size([22, 32])
torch.Size([32, 3, 224, 224])
torch.Size([22, 32])
torch.Size([32, 3, 224, 224])
torch.Size([21, 32])
torch.Size([32, 3, 224, 224])
torch.Size([24, 32])
torch.Size([32, 3, 224, 224])
torch.Size([21, 32])
torch.Size([32, 3, 224, 224])
torch.Size([31, 32])
torch.Size([32, 3, 224, 224])
torch.Size([20, 32])
torch.Size([32, 3, 224, 224])
torch.Size([23, 32])
torch.Size([32, 3, 224, 224])
torch.Size([26, 32])
torch.Size([32, 3, 224, 224])
torch.Size([20, 32])
torch.Size([32, 3, 224, 224])
torch.Size([24, 32])
torch.Size([32, 3, 224, 224])
torch.Size([24, 32])
torch.Size([32, 3, 224, 224])
t

# Модель

Модель подписей к изображениям состоит из кодировщика и декодера. В качестве кодировщика выступает линейный слой, который используется сразу после предобученной ***CNN***. Декодера - ***RNN***. 

<img src='https://raw.githubusercontent.com/yunjey/pytorch-tutorial/master/tutorials/03-advanced/image_captioning/png/model.png'>

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size, train_CNN=False):
        super(EncoderCNN, self).__init__()
        self.train_CNN = train_CNN
        # Загрузка предобученной CNN модели
        self.inception = models.inception_v3(pretrained=True, aux_logits=False)
        # Замена последнего слоя-классификатора на эмбеддер
        self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
        self.relu = nn.ReLU()
        self.times = []
        self.dropout = nn.Dropout(0.5)

    def forward(self, images):
        features = self.inception(images)
        return self.dropout(self.relu(features))


class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, features, captions):
        embeddings = self.dropout(self.embed(captions))
        embeddings = torch.cat((features.unsqueeze(0), embeddings), dim=0)
        hiddens, _ = self.lstm(embeddings)
        outputs = self.linear(hiddens)
        return outputs


class CNNtoRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size)
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions)
        return outputs

    def caption_image(self, image, vocabulary, max_length=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoderCNN(image).unsqueeze(0)
            states = None

            for _ in range(max_length):
                hiddens, states = self.decoderRNN.lstm(x, states)
                output = self.decoderRNN.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                result_caption.append(predicted.item())
                x = self.decoderRNN.embed(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_caption]

# Обучение модели

In [ ]:
def train():
    transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    train_loader, dataset = get_loader(
        root_folder="flickr8k/images",
        annotation_file="flickr8k/captions.txt",
        transform=transform,
        num_workers=2,
    )

    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_CNN = False

    # Гиперпараметры
    embed_size = 128
    hidden_size = 128
    vocab_size = len(dataset.vocab)
    num_layers = 1
    learning_rate = 3e-4
    num_epochs = 3


    # Инициализация
    model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Только тонкая настройка CNN
    for name, param in model.encoderCNN.inception.named_parameters():
        if "fc.weight" in name or "fc.bias" in name:
            param.requires_grad = True
        else:
            param.requires_grad = train_CNN

    model.train()

    for epoch in range(num_epochs):

        for idx, (imgs, captions) in tqdm(
            enumerate(train_loader), total=len(train_loader), leave=False
        ):
            imgs = imgs.to(device)
            captions = captions.to(device)

            outputs = model(imgs, captions[:-1])
            loss = criterion(
                outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
            )

            optimizer.zero_grad()
            loss.backward(loss)
            optimizer.step()

train()

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

  0%|          | 6/1265 [01:14<4:12:57, 12.06s/it]

# Домашнее задание

Проанализируйте, как размер эмбеддинга влияет на качество модели.

Визуализируйте полученный результат подписи к нескольким изображениям, можно ли улучшить производительность сети?